# Example Chapter 5: Using APIs to get Stock Data
In this example we use functions that tap into the yfinance library and Ken French's website to get monthly returns on the Vanguard S\&P 500 ETF (VOO) and short-term US t-bills. We use these data to estimate the annualized expected return, volatility, and Sharpe ratio for VOO. 

In [4]:
# Load in bates_boyer_fletcher_functions.py which 
!curl -O https://raw.githubusercontent.com/boyerb/Investments/refs/heads/master/functions/bates_boyer_fletcher_functions.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  4128  100  4128    0     0  52280      0 --:--:-- --:--:-- --:--:-- 55783


In [8]:

import bates_boyer_fletcher_functions as invf
import numpy as np

In [6]:
# Get monthly returns for a given ticker
ticker = ['VOO']
dat = invf.get_monthly_returns(ticker, '2020-01-01', '2023-12-31', tbill_return=False)
print(dat.head())

[*********************100%***********************]  1 of 1 completed

              VOO
Date             
2020-01 -0.000372
2020-02 -0.080997
2020-03 -0.124414
2020-04  0.127861
2020-05  0.047360



C:\Users\bhb35\Documents\data\investments\PycharmProjects\Investments\functions\bates_boyer_fletcher_functions.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  month_end_data = data.resample('M').ffill()  # 'ME' gives month-end, ffill to get last available price


In [ ]:
# Calcualte the annualized average return and volatility
A_Avg_VOO = dat.mean() * 12
A_Vol_VOO = dat.std() * np.sqrt(12)
print('Annuliazed Average: ', A_Avg_VOO)
print('Annualized Vol: ',  A_Vol_VOO)

In [ ]:
ticker = 'VOO'
ticker_list = [ticker]

dat = invf.get_monthly_returns(ticker_list, '1990-01-01', '2023-12-31', tbill_return=True)
print(dat.head())
dat['exret'] = dat[ticker] - dat['RF']
A_AVG_exret = dat['exret'].mean() * 12
A_Vol_exret = dat[ticker].std() * np.sqrt(12)
Sharpe = A_AVG_exret/A_Vol_exret
print()
print('Sharpe Ratio: ', Sharpe)